# Yield stress Model Selection

Notebook to create interactive app with function:
* Upload rheology xls file exported from trios with multitab option selected
* Execute Fit to flow curve data with model or models selected from a list

In [3]:
%matplotlib widget
import sys
sys.path.append("./../../")
import os
import ipywidgets as widgets
from IPython.display import clear_output
import pandas as pd
import matplotlib.pyplot as plt
import rheofit

# Select and upload data file (currently 1 file only)

In [4]:
import io
from ipyupload import FileUpload

w = FileUpload(accept=None,
               multiple=False, 
               disabled=False,
               style_button='color: darkblue; background-color: lightblue; width: 300px;',
               compress_level=9
              )
w

FileUpload(compress_level=9, style_button='color: darkblue; background-color: lightblue; width: 300px;')

# Select Rheological model or models from the list of available models

In [7]:
model_list=[rheofit.models.Bingham_model,
            rheofit.models.casson_model,
            rheofit.models.HB_model,
            rheofit.models.Powerlaw_model,
            rheofit.models.TC_model,
           ]

for model in model_list:
    print(model.name)
    display(model.model_expression)

Model(Bingham, prefix='bingham_')


<IPython.core.display.Math object>

Model(casson, prefix='casson_')


<IPython.core.display.Math object>

Model(HB)


<IPython.core.display.Math object>

Model(Powerlaw, prefix='PL_')


<IPython.core.display.Math object>

In [8]:
select_model_widget=widgets.SelectMultiple(options={item.name:item for item in model_list})
select_model_widget

SelectMultiple(options={"Model(Bingham, prefix='bingham_')": <lmfit.Model: Model(Bingham, prefix='bingham_')>,…

In [11]:
def execute_analysis(sender):
    
    figlist=plt.get_fignums()
    
    for i in figlist:
        plt.close(i)
    
    datafile_object=pd.ExcelFile(io.BytesIO(w.li_content[0]))
    table_name_list = datafile_object.sheet_names
    rheodata=datafile_object.parse(table_name_list[-1],skiprows=1).drop(0).reset_index().astype('float')
    
    condition=rheodata['Shear rate']<50
    
    data=rheodata[condition]
    
    
    clear_output()
    display(exec_analysis_button)
    
    global fit_list        
    fit_list=[rheofit.models.fit_FC(model,data) for model in select_model_widget.value]
   
    table_list=[rheofit.models.show_parameter_table(fit) for fit in fit_list];
    result_table=pd.concat(table_list,sort=False)
    
    
    display(result_table)
    
    for fit, model in zip(fit_list,select_model_widget.value):
        display(fit)
        plt.figure()
        rheofit.models.plot_fit_fc(fit,show_table=False)
        #fit.plot_residuals(yerr=False)
        #plt.xscale('log')
        #plt.yscale('symlog')
        #plt.ylim(-1000,1000)

exec_analysis_button=widgets.Button(description='Execute Analysis')
exec_analysis_button.on_click(execute_analysis)


In [12]:
display(exec_analysis_button)

Button(description='Execute Analysis', style=ButtonStyle())

,bingham_eta_bg,bingham_ystress,bic,redchi,casson_eta_bg,casson_ystress,n,K,ystress
"Model(Bingham, prefix='bingham_')",0.01423,0.027338,-98.350450,0.060948,NaN,NaN,NaN,NaN,NaN
"Model(casson, prefix='casson_')",NaN,NaN,-164.260544,0.010264,0.008004,0.020929,NaN,NaN,NaN
Model(HB),NaN,NaN,-286.020519,0.000357,NaN,NaN,0.578298,0.044922,0.017983


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …